# 1) Set up

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# rewind back to root
from os import path, chdir, listdir, getcwd
root_assets = 'content/drive/MyDrive/JACOBB'

In [3]:
# rewind back to root
from os import path, chdir, listdir
chdir('..')

In [4]:
# imports
import numpy as np
from ucr import load_UCR_dataset
from utils import plot_with_labels
from config import PATH_TO_DATA, PATH_TO_MODELS, PATH_TO_DATA_UCR, PATH_TO_MODELS_UCR
from itertools import compress
from models.models_utils import load_pretrained
from scikit_wrappers import CausalCNNEncoderClassifier
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE

%matplotlib inline

# 2) Dataset

In [5]:
# List datasets
complete_path = path.join(PATH_TO_DATA, PATH_TO_DATA_UCR)
ls_datasets = listdir( complete_path )
ls_datasets = [i_ for i_ in ls_datasets if path.isdir( path.join(complete_path, i_) )]

print(ls_datasets[:5])

ls_datasets = ['Adiac']
# Load dataset - UCR
id_ = 0
loaded_data = load_UCR_dataset(complete_path, ls_datasets[id_])

print('Loaded dataset: %s'%(ls_datasets[id_]))

['Haptics', 'SyntheticControl', 'Worms', 'Computers', 'HouseTwenty']
Loaded dataset: Adiac


In [6]:
print(loaded_data[0].shape)

(390, 1, 176)


In [7]:
# Split
dt_tr = loaded_data[0]
dt_tr_lab = loaded_data[1]
dt_ts = loaded_data[2]
dt_ts_lab = loaded_data[3]

In [8]:
print(dt_tr.shape)

(390, 1, 176)


# 3) Model : training

In [10]:
# Default hyper-params
path_to_hyper = "default_hyperparameters.json"

# Init encoder
model = load_pretrained(root_assets, model_origin='default')

In [11]:
model

CausalCNNEncoderClassifier(batch_size=10, channels=40, compared_length=inf,
              cuda=False, depth=10, early_stopping=None, gpu=False,
              in_channels=1, kernel_size=3, lr=0.001, nb_random_samples=10,
              nb_steps=1500, negative_penalty=1, out_channels=320,
              penalty=None, reduced_size=160)

In [12]:
# Encode dataset
model.fit_encoder(dt_tr)

CausalCNNEncoder(
  (network): Sequential(
    (0): CausalCNN(
      (network): Sequential(
        (0): CausalConvolutionBlock(
          (causal): Sequential(
            (0): Conv1d(1, 40, kernel_size=(3,), stride=(1,), padding=(2,))
            (1): Chomp1d()
            (2): LeakyReLU(negative_slope=0.01)
            (3): Conv1d(40, 40, kernel_size=(3,), stride=(1,), padding=(2,))
            (4): Chomp1d()
            (5): LeakyReLU(negative_slope=0.01)
          )
          (upordownsample): Conv1d(1, 40, kernel_size=(1,), stride=(1,))
        )
        (1): CausalConvolutionBlock(
          (causal): Sequential(
            (0): Conv1d(40, 40, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(2,))
            (1): Chomp1d()
            (2): LeakyReLU(negative_slope=0.01)
            (3): Conv1d(40, 40, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(2,))
            (4): Chomp1d()
            (5): LeakyReLU(negative_slope=0.01)
          )
        )
        (2): Cau

# 4) Model : inference

In [ ]:
# Encode sequences
enc_tr = model.encode(dt_tr)
enc_ts = model.encode(dt_ts)

In [ ]:
# Dimensionality reduction
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_tr = tsne.fit_transform(enc_tr)
tsne_ts = tsne.fit_transform(enc_ts)
print(tsne_tr.shape)

In [ ]:
# Display: training data
hF, Ax = plot_with_labels(tsne_tr, dt_tr_lab, marker='o',
                          legend='TRAIN', 
                          title='Dataset: %s'%(ls_datasets[id_]))

In [ ]:
# Display: test data
hF, Ax = plot_with_labels(tsne_ts, dt_ts_lab, marker='o',
                          legend='TEST', 
                          title='Dataset: %s'%(ls_datasets[id_]))